<a href="https://colab.research.google.com/github/iibragimov-beep/excel-converter-tool/blob/main/Avaya_ACCES_buttons_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 📞 Tool 1: Extract Buttons
import re
import openpyxl
from openpyxl.utils import get_column_letter
from google.colab import files
import io

def format_button(button):
    # (Your exact original logic)
    key_match = re.search(r'key:(\d+)', button)
    if not key_match:
        return None, None
    key = int(key_match.group(1))
    value_match = re.search(r'value:([^,|]+)', button)
    if not value_match:
        return None, None
    button_type = value_match.group(1)
    label_match = re.search(r'LabelName=([^;,\|]*)', button)
    label = label_match.group(1) if label_match else ""
    output = ""

    if button_type == 'vu-display':
        extension_match = re.search(r'Extension=([^;,\|]*)', button)
        if extension_match: output = f"vu-display={extension_match.group(1)}"
    elif button_type == 'autodial':
        dialed_match = re.search(r'DialedNumber=([^;,\|]*)', button)
        if dialed_match: output = f"autodial={dialed_match.group(1)}"
    elif button_type == 'q-calls':
        group_match = re.search(r'EmployeeGroup=(\d+)', button)
        if group_match: output = f"q-calls={group_match.group(1)}"
    elif button_type == 'busy-ind':
        bi_match = re.search(r'BIExtension=(\d+)', button)
        if bi_match: output = f"busy-ind={bi_match.group(1)}"
    elif button_type == 'aut-msg-wt':
        mwi_match = re.search(r'MWILampExtension=(\d+)', button)
        if mwi_match: output = f"aut-msg-wt={mwi_match.group(1)}"
    elif button_type == 'brdg-appr':
        ext_match = re.search(r'Ext=(\d+)', button)
        button_match = re.search(r'Button=([^;,\|]+)', button)
        if ext_match and button_match: output = f"brdg-appr={ext_match.group(1)},{button_match.group(1)}"
    elif button_type == 'sip-sobsrv':
        listen_match = re.search(r'ListenOnly=(true|false)', button)
        coach_match = re.search(r'Coach=(true|false)', button)
        components = []
        if listen_match and listen_match.group(1) == 'true': components.append("ListenOnly")
        if coach_match and coach_match.group(1) == 'true': components.append("Coach")
        if components: output = f"sip-sobsrv {' '.join(components)}"
    else:
        output = button_type

    if label: output = f"LabelName={label} {output}"
    return key, output

def get_max_keys(ws_input):
    max_key = 0
    for row in ws_input.iter_rows(min_row=2, values_only=True):
        button_features = row[6]
        if button_features:
            for button in button_features.split('|'):
                key_match = re.search(r'key:(\d+)', button)
                if key_match: max_key = max(max_key, int(key_match.group(1)))
    return max_key

def process_input_excel(input_file, output_file):
    wb_input = openpyxl.load_workbook(input_file)
    ws_input = wb_input['Endpoints']
    max_keys = get_max_keys(ws_input) or 52
    wb_output = openpyxl.Workbook()
    ws_output = wb_output.active
    ws_output.title = "Avaya Buttons"
    headers = (['Name', 'Number', 'PermissionSet'] + [f'Key {i}' for i in range(1, max_keys + 1)] + ['Profile', 'GroupId', 'BridgedCallAlerting', 'DialingOption', 'HeadsetSignaling', 'ButtonClicks', 'PhoneScreen', 'Redial', 'AudioPath', 'UserPrefferedLanguage'])

    for col, header in enumerate(headers, start=1):
        ws_output.cell(row=1, column=col, value=header)

    for row_idx, row in enumerate(ws_input.iter_rows(min_row=2, values_only=True), start=2):
        name, number, permission_set = row[0], row[1], row[5]
        button_features, profile, group_id = row[6], row[7], row[13]
        bridged_call_alerting, dialing_option = row[15], row[18]
        headset_signaling, audio_path, button_clicks = row[19], row[20], row[21]
        phone_screen, user_pref_lang, redial = row[22], row[26], row[28]

        ws_output[f"A{row_idx}"] = name
        ws_output[f"B{row_idx}"] = number
        ws_output[f"C{row_idx}"] = permission_set
        ws_output[f"{get_column_letter(max_keys + 4)}{row_idx}"] = profile
        ws_output[f"{get_column_letter(max_keys + 5)}{row_idx}"] = group_id
        ws_output[f"{get_column_letter(max_keys + 6)}{row_idx}"] = bridged_call_alerting
        ws_output[f"{get_column_letter(max_keys + 7)}{row_idx}"] = dialing_option
        ws_output[f"{get_column_letter(max_keys + 8)}{row_idx}"] = headset_signaling
        ws_output[f"{get_column_letter(max_keys + 9)}{row_idx}"] = button_clicks
        ws_output[f"{get_column_letter(max_keys + 10)}{row_idx}"] = phone_screen
        ws_output[f"{get_column_letter(max_keys + 11)}{row_idx}"] = redial
        ws_output[f"{get_column_letter(max_keys + 12)}{row_idx}"] = audio_path
        ws_output[f"{get_column_letter(max_keys + 13)}{row_idx}"] = user_pref_lang

        if button_features:
            for button in button_features.split('|'):
                key, output = format_button(button)
                if key and output is not None and key <= max_keys:
                    col = get_column_letter(key + 3)
                    ws_output[f"{col}{row_idx}"] = output

    wb_output.save(output_file)

# --- COLAB USER INTERFACE ---
print("=========================================")
print("📞 TOOL 1: EXTRACT BUTTONS")
print("Upload your raw 'Endpoints' Excel file below.")
print("=========================================")

uploaded = files.upload()

for file_name in uploaded.keys():
    print("Processing file... Please wait.")
    input_file = io.BytesIO(uploaded[file_name])
    output_filename = "Avaya_Buttons_Output.xlsx"

    process_input_excel(input_file, output_filename)

    print("✅ Success! Downloading your formatted file...")
    files.download(output_filename)

📞 TOOL 1: EXTRACT BUTTONS
Upload your raw 'Endpoints' Excel file below.


In [ ]:
#@title 🔄 Tool 2: Reconstruct ACCEC Import File
import openpyxl
import re
from google.colab import files
import io

def reverse_format(simplified, key, display_format=9):
    # (Your exact original logic)
    simplified = (simplified or "").strip()
    if not simplified: return None
    if "logout-ovr" in simplified:
        label = simplified[len("LabelName="):].replace("logout-ovr", "").strip() if simplified.startswith("LabelName=") else ""
        return f"params:LabelName={label},key:{key},value:logout-ovr"

    label_value = ""
    rest = simplified

    if simplified.startswith("LabelName="):
        known_types = r'\b(vu-display|autodial|q-calls|busy-ind|aut-msg-wt|brdg-appr|sip-sobsrv|call-appr|call-fwd|send-calls|agnt-login|auto-in|aux-work|after-call|mct-act|call-park|call-unpk|call-pkup)\b'
        match = re.search(known_types, simplified)
        if match:
            label_end = match.start()
            label_value = simplified[len("LabelName="):label_end].rstrip('; ').strip()
            rest = simplified[label_end:].strip()
        else:
            label_value = simplified[len("LabelName="):].rstrip('; ').strip()
            rest = ""

    if "sip-sobsrv" in rest:
        label = label_value or ""
        return f"params:ListenOnly=true;Coach=false;LabelName={label},key:{key},value:sip-sobsrv"

    if '=' in rest:
        button_type, param = rest.split('=', 1)
        param = param.strip()
    else:
        button_type = rest.strip()

    if button_type == "busy-ind" and param: return f"params:LabelName={label_value or ''};BIExtension={param},key:{key},value:busy-ind"
    if button_type == "vu-display" and param: return f"params:LabelName={label_value or ''};DisplayFormat={display_format};Extension={param},key:{key},value:vu-display"
    if button_type == "autodial" and param and label_value == "Overhead Paging": return f"params:DialedNumber={param};LabelName={label_value},key:{key},value:autodial"
    if button_type == "autodial" and param: return f"params:LabelName={label_value or ''};DialedNumber={param},key:{key},value:autodial"
    if button_type == "aut-msg-wt" and param: return f"params:LabelName={label_value or ''};MWILampExtension={param},key:{key},value:aut-msg-wt"
    if button_type == "brdg-appr" and param:
        parts = param.split(",")
        if len(parts) == 2: return f"params:LabelName={label_value or ''};Button={parts[1]};Ext={parts[0]},key:{key},value:brdg-appr"
    if button_type == "q-calls" and param: return f"params:LabelName={label_value or ''};EmployeeGroup={param},key:{key},value:q-calls"

    params_body = "Tone=half-ring;RingerType=no-ring;" if button_type == "call-pkup" else ""
    label_part = f"LabelName={label_value}," if label_value else "LabelName=,"

    full = f"params:{params_body}{label_part}key:{key},value:{button_type}" if button_type == "call-pkup" else f"params:{label_part}{params_body}key:{key},value:{button_type}"
    full = re.sub(r';+', ';', full)
    full = re.sub(r';key:', 'key:', full)
    return full.rstrip(';')

def reconstruct_input(input_data, reconstructed_file):
    wb = openpyxl.load_workbook(input_data)
    ws = wb['Avaya Buttons']
    headers = [cell.value for cell in ws[1] if cell.value]
    key_headers = [h for h in headers if h.startswith("Key ")]
    max_keys = max(int(h.split()[1]) for h in key_headers)

    original_headers = ["Name", "Number", "AgentPhone", "Password", "AutoAnswer", "PermissionSet", "ButtonFeatures", "Profile", "VmNumber", "Room", "Floor", "Building", "CoveragePathId", "GroupId", "MultiRegistration", "BridgedCallAlerting", "BlockEnhancedCallPickupAlerting", "PhoneScreenOnCalling", "DialingOption", "HeadsetSignaling", "AudioPath", "ButtonClicks", "PhoneScreen", "BackgroundLogo", "PersonalizedRinging", "InternalCallCdr", "UserPrefferedLanguage", "TimeFormat", "Redial"]

    wb_new = openpyxl.Workbook()
    ws_new = wb_new.active
    ws_new.title = "Endpoints"

    for col, h in enumerate(original_headers, start=1):
        ws_new.cell(row=1, column=col, value=h)

    header_map = {h: i + 1 for i, h in enumerate(headers)}

    for row_idx in range(2, ws.max_row + 1):
        row = [""] * len(original_headers)
        field_map = {"Name": 0, "Number": 1, "PermissionSet": 5, "Profile": 7, "GroupId": 13, "BridgedCallAlerting": 15, "DialingOption": 18, "HeadsetSignaling": 19, "ButtonClicks": 21, "PhoneScreen": 22, "Redial": 28}

        for field, idx in field_map.items():
            if field in header_map: row[idx] = ws.cell(row=row_idx, column=header_map[field]).value

        button_parts = []
        vu_display_count = 0
        for k in range(1, max_keys + 1):
            col = header_map.get(f"Key {k}")
            if not col: continue
            simplified = ws.cell(row=row_idx, column=col).value
            if not simplified: continue

            if "vu-display" in simplified:
                vu_display_count += 1
                part = reverse_format(simplified, k, 8 + vu_display_count)
            else:
                part = reverse_format(simplified, k)
            if part: button_parts.append(part)

        row[6] = "|".join(button_parts)
        row[2], row[3], row[8], row[12], row[14], row[16], row[17], row[24], row[25], row[27] = "Local", "24688", "59030", "IXM", "1", "TRUE", "false", "CLASSIC_TONE_3", "USE_PROFILE", "HOUR_12"

        for col, val in enumerate(row, start=1):
            ws_new.cell(row=row_idx, column=col, value=val)

    wb_new.save(reconstructed_file)

# --- COLAB USER INTERFACE ---
print("=========================================")
print("🔄 TOOL 2: RECONSTRUCT ACCEC")
print("Upload your modified 'Avaya Buttons' Excel file below.")
print("=========================================")

uploaded = files.upload()

for file_name in uploaded.keys():
    print("Reconstructing file... Please wait.")
    input_data = io.BytesIO(uploaded[file_name])
    output_filename = "ACCEC_Input_File.xlsx"

    reconstruct_input(input_data, output_filename)

    print("✅ Success! Downloading your ACCEC import file...")
    files.download(output_filename)

🔄 TOOL 2: RECONSTRUCT ACCEC
Upload your modified 'Avaya Buttons' Excel file below.


Saving Avaya_Buttons_Output.xlsx to Avaya_Buttons_Output (1).xlsx
Reconstructing file... Please wait.
✅ Success! Downloading your ACCEC import file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>